## 【コロモゴロフスミルノフ検定（KS検定）】

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage

from tqdm import tqdm
import time
import sys

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [2]:
import glob
win_path = f'../features/4_winner/*.gz'
#  Data Load
base = utils.read_df_pkl('../input/base*')
win_path_list = glob.glob(win_path)
# tmp_path_listには検証中のfeatureを入れてある
tmp_path_list = glob.glob('../features/5_tmp/*.gz')
#  tmp_path_list = glob.glob('../season1_features/features/5_tmp/*.gz')
win_path_list += tmp_path_list

train_path_list = []
test_path_list = []
for path in win_path_list:
    if path.count('train'):
        train_path_list.append(path)
    elif path.count('test'):
        test_path_list.append(path)

base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)
train_feature_list = utils.parallel_load_data(path_list=train_path_list)
test_feature_list = utils.parallel_load_data(path_list=test_path_list)
train = pd.concat(train_feature_list, axis=1)
train = pd.concat([base_train, train], axis=1)
test = pd.concat(test_feature_list, axis=1)
test = pd.concat([base_test, test], axis=1)

100%|██████████| 3/3 [00:00<00:00, 39.59it/s]


In [7]:
from scipy.stats import ks_2samp
list_p_value =[]

for col in tqdm(train.columns):
    list_p_value.append(ks_2samp(test[col] , train[col])[1])

se_p = pd.Series(list_p_value, index = train.columns).sort_values() 
list_discarded = se_p[se_p < .1]
list_discarded = set( list_discarded)
se_p[se_p < .1]

165_fe3_lag0_-2_auth0_cat1_unix_date_min@                    0.000000e+00
166_fe3_new_cat0_month_diff_mean@                            0.000000e+00
167_fe3_lag-3_-13_auth1_cat0_month_diff_std@                 0.000000e+00
167_fe3_lag-3_-13_auth1_cat1_unix_date_max@                  0.000000e+00
165_fe3_lag0_-2_auth1_cat0_month_diff_std@                   0.000000e+00
165_fe3_lag0_-2_auth0_cat1_month_diff_mean@                  0.000000e+00
165_fe3_lag0_-2_auth0_cat0_unix_date_min@                    0.000000e+00
165_fe3_lag0_-2_auth0_cat1_unix_date_max@                    0.000000e+00
165_fe3_lag0_-2_auth1_cat1_month_diff_mean@                  0.000000e+00
167_fe3_lag-3_-13_auth0_cat1_month_diff_std@                 0.000000e+00
167_fe3_lag-3_-13_auth0_cat1_unix_date_min@                  0.000000e+00
167_fe3_lag-3_-13_auth1_cat0_unix_date_min@                  0.000000e+00
167_fe3_lag-3_-13_auth1_cat1_month_diff_std@                 0.000000e+00
167_fe3_lag-3_-13_auth0_cat0_unix_date

In [4]:
list_discarded

{'165_fe3_lag0_-2_auth0_cat0_month_diff_mean@',
 '165_fe3_lag0_-2_auth0_cat0_month_diff_std@',
 '165_fe3_lag0_-2_auth0_cat0_unix_date_max@',
 '165_fe3_lag0_-2_auth0_cat0_unix_date_min@',
 '165_fe3_lag0_-2_auth0_cat1_month_diff_mean@',
 '165_fe3_lag0_-2_auth0_cat1_month_diff_std@',
 '165_fe3_lag0_-2_auth0_cat1_unix_date_max@',
 '165_fe3_lag0_-2_auth0_cat1_unix_date_min@',
 '165_fe3_lag0_-2_auth1_cat0_dummie_category_2_3_0_mean@',
 '165_fe3_lag0_-2_auth1_cat0_month_diff_mean@',
 '165_fe3_lag0_-2_auth1_cat0_month_diff_std@',
 '165_fe3_lag0_-2_auth1_cat0_purchase_amount_mean@',
 '165_fe3_lag0_-2_auth1_cat0_unix_date_max@',
 '165_fe3_lag0_-2_auth1_cat0_unix_date_min@',
 '165_fe3_lag0_-2_auth1_cat1_month_diff_mean@',
 '165_fe3_lag0_-2_auth1_cat1_month_diff_std@',
 '165_fe3_lag0_-2_auth1_cat1_unix_date_max@',
 '165_fe3_lag0_-2_auth1_cat1_unix_date_min@',
 '166_fe3_new_cat0_dummie_category_2_1_0_mean@',
 '166_fe3_new_cat0_month_diff_mean@',
 '166_fe3_new_cat0_month_diff_std@',
 '166_fe3_new_ca

In [6]:
import shutil
import re
import os

list_discarded = list(set( list_discarded))
for feat in list_discarded:
    if feat.count('target'):continue
    train_path = f'{feat[:8]}train_{feat[8:]}.gz'.replace('2_1_0', '2_1.0').replace('2_3_0', '2_3.0')
    test_path = f'{feat[:8]}test_{feat[8:]}.gz'.replace('2_1_0', '2_1.0').replace('2_3_0', '2_3.0')
    if  os.path.exists(f'../features/3_third_valid/{train_path}'):
        continue
    if  os.path.exists(f'../features/3_third_valid/{test_path}'):
        continue
    shutil.move(f'../features/4_winner/{train_path}', '../features/3_third_valid/')
    shutil.move(f'../features/4_winner/{test_path}', '../features/3_third_valid/')